# Python Challenge using Genetic Computing

## 8 Queen challenge Solved using Genetic Algorithm

## Description

The eight queens puzzle is the problem of placing eight chess queens on an 8×8 chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. The eight queens puzzle is an example of the more general n queens problem of placing n non-attacking queens on an n×n chessboard. (Source : https://en.wikipedia.org/wiki/Eight_queens_puzzle )

## Challenge

The challenge is to generate one right sequence through Genetic Programming. The sequence has to be 8 numbers between 0 to 7. Each number represents the positions the Queens can be placed. Each number refers to the row number in the specific column

0 3 4 5 6 1 2 4

· 0 is the row number in the column 0 where the Queen can be placed

· 3 is the row number in the column 1 where the Queen can be place

In [1]:
import numpy as np
import pandas as pd
import unittest
from array import *
from numpy import random
from math import ceil

### Defining Constants and population_initialization function

In [2]:
noOfQueens = 8
mutationRate = 0.1
Totalpopulation = 200
crossOver = 0.5
generationCount = 1000

def population_initialization(populations, scores):
    for count in range(Totalpopulation):
        positions = np.random.randint(noOfQueens,size=noOfQueens)
        populations.append(positions)
        scores.append(fitness_score(positions))
    

### Fitness Score function

In [3]:
#This function calculates the score for the queens position in a chess board
#The score is calculated based on the number of possible attacks for a combination
#   a. Row wise check: This checks whether any queen poistioned on same row.
#   b. diagonal check: This checks whether any queen positioned at any of its diagonal positions.
#The score is zero for a combination where there are no possbile attacks
#The non zero score denotes the number of times queen attacks is possible.

def fitness_score(population):
    score = 0
    #Row Wise check
    for i in range(noOfQueens):     
        if i!=noOfQueens-1:
            for j in range(i+1,noOfQueens):               
                if population[i] == population[j]:
                    score+=1
    #Calculate the diagonals for a queen position
    population_xy = [(population[y],y) for y in range(noOfQueens) ]    
    diagonal=[]
    for item in population_xy:
        x=item[0]
        y = z = item[1]
        diagonal_item=[]
        for x1 in reversed(range(x)):
            y=y+1
            if (y<=noOfQueens-1):
                diagonal_item.append((x1,y))
        for x2 in range(x+1,noOfQueens):
            z=z+1
            if (z<=noOfQueens-1):
                diagonal_item.append((x2,z))
        diagonal.append(diagonal_item)    
    #b check if any queen present diagonally
    for item in range(noOfQueens):
        for queen in diagonal[item]:
            if queen in population_xy:
                #print("queen attacks")
                score += 1
    return score


### Cross Over Function

In [5]:
#Function to create two new childs from 2 best parents.
#Example: parent1: [0 6 3 5 0 2 4 6] parent2: [4 0 5 5 7 2 6 3]
#         child1: [0 6 3 5 7 2 6 3] child2: [4 0 5 5 0 2 4 6]
def crossover(draw):   
    crossOverPoint = int(crossOver * noOfQueens)    
    child = [np.concatenate([draw[0][0:crossOverPoint], draw[1][crossOverPoint:]]) ,
           np.concatenate([draw[1][0:crossOverPoint], draw[0][crossOverPoint:]])]
    return child

### Mutation Function

In [6]:
#Function to change the random property of the child by replacing it with ranom value
#Example: Child : [0 6 3 5 7 2 6 3]
#Child after mutation: [0 6 3 5 4 2 6 3]
def mutation(child_for_mutation):
    mutation_bits = int(ceil(mutationRate * noOfQueens))
    #print("Before mutation :",child_for_mutation)
    for i in range(0, mutation_bits):
        child_for_mutation[0][random.randint(0,noOfQueens)] = random.randint(0,noOfQueens)
        child_for_mutation[1][random.randint(0,noOfQueens)] = random.randint(0,noOfQueens)   
    return child_for_mutation 

### Main program

In [10]:
#####main program
####Step 1: Initialize the population with random positions and fitness scores
populationData=[]
fitness_scores=[]
population_initialization(populationData, fitness_scores)
#create a data frame
columns = {"QueenPositions":populationData, "FitnessScore":fitness_scores}  
queen_pos_score_df = pd.DataFrame(columns)
queen_pos_score_df = queen_pos_score_df.sort_values("FitnessScore")

####Step2: For each generation generate 2 childs from 2 best parents after cross over and mutation
for loop in range(generationCount):
    draw = [queen_pos_score_df[0:1]["QueenPositions"].values[0], queen_pos_score_df[1:2]["QueenPositions"].values[0]]
    #print('Fitness Scores of Parents ',get_fitness_score(draw[0]),get_fitness_score(draw[1]))
    if (fitness_score(draw[0])==0):
        print(draw[0])
        break
    elif (fitness_score(draw[1])==0):
        print(draw(1))
        break
    child = crossover(draw)
    mutated_child = mutation(child)
    #Append child and its fitness score to the array    
    mutated_child_ar = [np.array(mutated_child[0]), np.array(mutated_child[1])]
    populationData.extend(mutated_child_ar)
    mutated_child_scores = [fitness_score(mutated_child[0]),fitness_score(mutated_child[1])]
    fitness_scores.extend(mutated_child_scores)
    child_df = pd.DataFrame({'QueenPositions': mutated_child_ar,
                             'FitnessScore': mutated_child_scores})
    #Append child dataframe to the existing queen_pos_score_df
    queen_pos_score_df = queen_pos_score_df.append(child_df, ignore_index=True)
    queen_pos_score_df = queen_pos_score_df.sort_values("FitnessScore")
    queen_pos_score_df.reset_index(drop=True)
    print("Generation : {}, Average Score : {:5.2f}, BestFit : {}  {}" 
          .format(loop, queen_pos_score_df["FitnessScore"].mean(), draw[0], draw[1]))

Generation : 0, Average Score :  7.88, BestFit : [5 0 5 1 6 7 7 4]  [4 0 1 5 5 3 6 4]
Generation : 1, Average Score :  7.85, BestFit : [5 0 5 1 6 7 7 4]  [4 0 1 5 5 3 6 4]
Generation : 2, Average Score :  7.83, BestFit : [5 0 5 1 6 7 7 4]  [4 0 1 5 5 3 6 4]
Generation : 3, Average Score :  7.82, BestFit : [5 0 5 1 6 7 7 4]  [4 0 1 5 5 3 6 4]
Generation : 4, Average Score :  7.80, BestFit : [5 0 5 1 6 7 7 4]  [5 0 5 1 5 3 6 4]
Generation : 5, Average Score :  7.80, BestFit : [5 0 5 1 6 7 7 4]  [0 2 6 4 2 4 5 3]
Generation : 6, Average Score :  7.82, BestFit : [5 0 5 1 6 7 7 4]  [7 0 1 7 2 6 2 5]
Generation : 7, Average Score :  7.80, BestFit : [5 0 5 1 6 7 7 4]  [4 1 1 4 0 7 3 6]
Generation : 8, Average Score :  7.75, BestFit : [5 0 5 1 6 7 7 4]  [5 0 5 1 0 7 3 6]
Generation : 9, Average Score :  7.70, BestFit : [5 0 5 1 4 7 3 6]  [5 0 5 1 6 7 7 4]
Generation : 10, Average Score :  7.67, BestFit : [5 0 5 1 4 7 3 6]  [5 0 5 1 6 3 7 4]
Generation : 11, Average Score :  7.64, BestFit : [5 

Generation : 139, Average Score :  5.36, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 140, Average Score :  5.35, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 141, Average Score :  5.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 142, Average Score :  5.33, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 143, Average Score :  5.32, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 144, Average Score :  5.31, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 145, Average Score :  5.30, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 146, Average Score :  5.29, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 147, Average Score :  5.28, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 148, Average Score :  5.27, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 149, Average Score :  5.26, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 150, Average Score 

Generation : 270, Average Score :  4.40, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 271, Average Score :  4.40, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 272, Average Score :  4.39, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 273, Average Score :  4.39, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 274, Average Score :  4.39, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 275, Average Score :  4.39, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 276, Average Score :  4.38, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 277, Average Score :  4.38, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 278, Average Score :  4.38, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 279, Average Score :  4.37, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 280, Average Score :  4.36, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 3]
Generation : 281, Average Score 

Generation : 399, Average Score :  3.98, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 400, Average Score :  3.98, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 401, Average Score :  3.98, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 6]
Generation : 402, Average Score :  3.97, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 6]
Generation : 403, Average Score :  3.97, BestFit : [5 2 5 1 4 7 3 6]  [0 2 5 1 4 7 3 6]
Generation : 404, Average Score :  3.97, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 405, Average Score :  3.96, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 406, Average Score :  3.96, BestFit : [5 2 5 1 4 7 3 6]  [0 2 5 1 4 7 3 6]
Generation : 407, Average Score :  3.96, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 408, Average Score :  3.95, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 409, Average Score :  3.95, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 410, Average Score 

Generation : 518, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 519, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 520, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 521, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 522, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 523, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 524, Average Score :  3.73, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 525, Average Score :  3.72, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 526, Average Score :  3.72, BestFit : [5 2 5 1 4 7 3 6]  [0 2 5 1 4 7 3 6]
Generation : 527, Average Score :  3.72, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 528, Average Score :  3.72, BestFit : [5 2 5 1 4 7 3 6]  [0 2 5 1 4 7 3 6]
Generation : 529, Average Score 

Generation : 637, Average Score :  3.60, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 638, Average Score :  3.60, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 639, Average Score :  3.60, BestFit : [5 2 5 1 4 7 3 6]  [0 2 5 1 4 7 3 6]
Generation : 640, Average Score :  3.60, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 641, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 642, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 643, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 644, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 645, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 646, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 647, Average Score :  3.59, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 648, Average Score 

Generation : 751, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 752, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 753, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 754, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 755, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 756, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 757, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 758, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 759, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 760, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 6]
Generation : 761, Average Score :  3.49, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 762, Average Score 

Generation : 867, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [6 2 5 1 4 7 3 6]
Generation : 868, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 869, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 870, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 871, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 872, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 873, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 874, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 875, Average Score :  3.41, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 876, Average Score :  3.40, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 877, Average Score :  3.40, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 878, Average Score 

Generation : 982, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 983, Average Score :  3.35, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 984, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 0 6]
Generation : 985, Average Score :  3.35, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 986, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 987, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 988, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 989, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 990, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 991, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 0 1 4 7 3 6]
Generation : 992, Average Score :  3.34, BestFit : [5 2 5 1 4 7 3 6]  [5 2 5 1 4 7 3 6]
Generation : 993, Average Score 

In [12]:
fitness_score([3, 1, 6, 4, 0, 7, 5, 2])

0

In [14]:
class TestNQueen(unittest.TestCase):
    
    def test_get_fitness_score(self):
        result = fitness_score([1, 6, 5, 3, 5, 2, 2, 6])
        self.assertEqual(result, 7)
        result = fitness_score([3, 1, 6, 4, 0, 7, 5, 2])
        self.assertEqual(result, 0)
    
    def test_crossover(self):
        parent = [np.array([1, 3, 1, 4, 4, 7, 5, 2]), np.array([6, 4, 6, 7, 0, 7, 5, 6])]
        child = [np.array([1, 3, 1, 4, 0, 7, 5, 6]), np.array([6, 4, 6, 7, 4, 7, 5, 2])]
        result = crossover(parent)
        np.testing.assert_array_equal(result, child)
        
    def test_mutation(self):
        child = [np.array([1, 3, 1, 4, 0, 7, 5, 6]), np.array([6, 4, 6, 7, 4, 7, 5, 2])]
        result = mutation(child)
        print(result)
        np.testing.assert_raises(AssertionError, np.testing.assert_array_equal(result,child))
        
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)      

..D:\Anaconda\lib\site-packages\numpy\testing\nose_tools\utils.py:1191: DeprecationWarning: callable is None
  return nose.tools.assert_raises(*args,**kwargs)
.

[array([1, 3, 1, 4, 0, 2, 5, 6]), array([6, 4, 6, 7, 4, 7, 4, 2])]



----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK
